In [1]:
import duckdb
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
# Connect to an in-memory DuckDB database
conn = duckdb.connect()

In [4]:
df_sell_in = conn.query("SELECT DISTINCT * FROM read_csv_auto('../data/sell-in.txt')").to_df()
df_productos = conn.query("SELECT DISTINCT * FROM read_csv_auto('../data/tb_productos.txt')").to_df()
df_stocks = conn.query("SELECT DISTINCT * FROM read_csv_auto('../data/tb_stocks.txt')").to_df()
df_to_predict = conn.query("SELECT DISTINCT * FROM read_csv_auto('../data/product_id_to_predict_201912.txt')").to_df()

conn.register('sell_in', df_sell_in)
conn.register('products', df_productos)
conn.register('stocks', df_stocks)
conn.register('to_predict', df_to_predict)

In [5]:
# Print the df heads
print("df_sell_in")
df_sell_in

df_sell_in


,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10449,20654,0,1,0.00190,0.00190
1,201701,10065,20828,0,1,0.00460,0.00460
2,201701,10179,20828,0,1,0.00308,0.00308
3,201701,10004,20828,0,9,0.13056,0.13056
4,201701,10177,20828,0,1,0.01536,0.01536
...,...,...,...,...,...,...,...
2945813,201912,10008,20086,0,15,11.34166,11.34166
2945814,201912,10028,20086,0,1,0.21199,0.21199
2945815,201912,10124,20086,0,2,0.07571,0.07571
2945816,201912,10372,20086,0,1,0.03028,0.03028


In [990]:
print("df_productos")
df_productos

df_productos


,cat1,cat2,cat3,brand,sku_size,product_id
0,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20222
1,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20507
2,HC,ROPA LAVADO,Polvo,LIMPIEX,800,20455
3,HC,ROPA LAVADO,Polvo,LIMPIEX,3000,20024
4,HC,ROPA LAVADO,Liquido,LIMPIEX,400,20439
...,...,...,...,...,...,...
1248,HC,VAJILLA,Opaco,Importado,1000,20388
1249,PC,CABELLO,ACONDICIONADOR,SHAMPOO1,400,20722
1250,PC,CABELLO,ACONDICIONADOR,SHAMPOO1,750,20726
1251,PC,CABELLO,SHAMPOO,SHAMPOO1,400,20514


In [991]:
df_productos.isna().sum()

cat1          0
cat2          0
cat3          0
brand         0
sku_size      0
product_id    0
dtype: int64

In [992]:
print("df_stocks")
df_stocks

df_stocks


,periodo,product_id,stock_final
0,201810,21005,1.01338
1,201810,20803,0.86351
2,201810,20007,0.34054
3,201810,20744,12.79966
4,201810,20352,15.60962
...,...,...,...
13686,201912,20669,3.32964
13687,201912,21264,0.09653
13688,201912,20567,6.52317
13689,201912,20409,9.13185


In [993]:
print("df_to_predict")
df_to_predict

df_to_predict


,product_id
0,20017
1,20043
2,20049
3,20053
4,20054
...,...
775,21088
776,21114
777,21142
778,21192


In [6]:
productos_ventas = set(df_sell_in['product_id'].unique())
productos_maestro = set(df_productos['product_id'].unique())

faltantes_en_productos = productos_ventas - productos_maestro
print(f"🛑 Productos en df_sell_in pero NO en df_productos: {len(faltantes_en_productos)}")
print(faltantes_en_productos)

🛑 Productos en df_sell_in pero NO en df_productos: 45
{np.int64(21249), np.int64(21125), np.int64(21253), np.int64(21268), np.int64(21270), np.int64(21272), np.int64(21273), np.int64(21274), np.int64(21275), np.int64(21277), np.int64(21278), np.int64(21279), np.int64(21281), np.int64(21283), np.int64(21284), np.int64(21285), np.int64(21286), np.int64(21288), np.int64(21289), np.int64(21290), np.int64(21291), np.int64(21292), np.int64(21165), np.int64(21293), np.int64(21295), np.int64(21296), np.int64(21169), np.int64(21297), np.int64(21298), np.int64(21299), np.int64(20918), np.int64(21178), np.int64(20808), np.int64(21066), np.int64(21199), np.int64(21217), np.int64(21223), np.int64(21225), np.int64(21098), np.int64(21228), np.int64(21230), np.int64(20848), np.int64(21238), np.int64(21240), np.int64(21241)}


In [7]:
productos_a_predecir = set(df_to_predict['product_id'].unique())

faltantes_en_maestro_desde_to_predict = productos_a_predecir - productos_maestro
print(f"⚠️ Productos en df_to_predict pero NO en df_productos: {len(faltantes_en_maestro_desde_to_predict)}")
print(faltantes_en_maestro_desde_to_predict)

⚠️ Productos en df_to_predict pero NO en df_productos: 0
set()


In [8]:
# Conjuntos base
productos_ventas = set(df_sell_in['product_id'].unique())
productos_maestro = set(df_productos['product_id'].unique())
productos_a_predecir = set(df_to_predict['product_id'].unique())

# Productos en ventas pero no en el maestro
productos_faltantes_en_maestro = productos_ventas - productos_maestro

# Verificar si alguno de esos está en los productos a predecir
interseccion = productos_faltantes_en_maestro.intersection(productos_a_predecir)

print(f"🔍 Productos que están en df_sell_in pero NO en df_productos y SÍ en df_to_predict: {len(interseccion)}")
print(interseccion)


🔍 Productos que están en df_sell_in pero NO en df_productos y SÍ en df_to_predict: 0
set()


In [9]:
# Paso 1: Agrupar por product_id y periodo (sumando tn)
df_sell_in_grouped = df_sell_in.groupby(['product_id', 'periodo'])['tn'].sum().reset_index()
df_sell_in_grouped

,product_id,periodo,tn
0,20001,201701,934.77222
1,20001,201702,798.01620
2,20001,201703,1303.35771
3,20001,201704,1069.96130
4,20001,201705,1502.20132
...,...,...,...
31238,21295,201701,0.00699
31239,21296,201708,0.00651
31240,21297,201701,0.00579
31241,21298,201708,0.00573


In [10]:
# Paso 2: Crear la clase = tn del periodo + 2
df_sell_in_grouped = df_sell_in_grouped.sort_values(['product_id', 'periodo'])
df_sell_in_grouped['clase'] = df_sell_in_grouped.groupby('product_id')['tn'].shift(-2)
df_sell_in_grouped

,product_id,periodo,tn,clase
0,20001,201701,934.77222,1303.35771
1,20001,201702,798.01620,1069.96130
2,20001,201703,1303.35771,1502.20132
3,20001,201704,1069.96130,1520.06539
4,20001,201705,1502.20132,1030.67391
...,...,...,...,...
31238,21295,201701,0.00699,NaN
31239,21296,201708,0.00651,NaN
31240,21297,201701,0.00579,NaN
31241,21298,201708,0.00573,NaN


In [11]:
# Paso 3: Calcular lags tn_1 a tn_11
for i in range(1, 12):
    df_sell_in_grouped[f'tn_{i}'] = df_sell_in_grouped.groupby('product_id')['tn'].shift(i)
df_sell_in_grouped

,product_id,periodo,tn,clase,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11
0,20001,201701,934.77222,1303.35771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,201702,798.01620,1069.96130,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20001,201703,1303.35771,1502.20132,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20001,201704,1069.96130,1520.06539,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20001,201705,1502.20132,1030.67391,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31238,21295,201701,0.00699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31239,21296,201708,0.00651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31240,21297,201701,0.00579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31241,21298,201708,0.00573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Paso 4: Agregar columna tn_actual
# (es solo por claridad: la tn del mes actual)
df_sell_in_grouped['tn_actual'] = df_sell_in_grouped['tn']
df_sell_in_grouped

,product_id,periodo,tn,clase,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11,tn_actual
0,20001,201701,934.77222,1303.35771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,934.77222
1,20001,201702,798.01620,1069.96130,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,798.01620
2,20001,201703,1303.35771,1502.20132,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1303.35771
3,20001,201704,1069.96130,1520.06539,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1069.96130
4,20001,201705,1502.20132,1030.67391,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1502.20132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31238,21295,201701,0.00699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00699
31239,21296,201708,0.00651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00651
31240,21297,201701,0.00579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00579
31241,21298,201708,0.00573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00573


In [13]:
#Experimento con pesos para la regression lineal
lags = ['tn', 'tn_1', 'tn_2', 'tn_3', 'tn_4', 'tn_5', 'tn_6', 'tn_7', 'tn_8', 'tn_9', 'tn_10', 'tn_11']

#Opcion 1: Volumen promedio del producto (tn o media de los 12 meses)
#Esta es la opcion que mejor anduvo
df_sell_in_grouped['peso1'] = df_sell_in_grouped[lags].mean(axis=1)

#Opcion 2: Varianza de las ventas históricas
varianzas = df_sell_in_grouped[lags].var(axis=1)
# Reemplazamos NaN por una varianza alta (para que el peso sea bajo)
varianzas.fillna(varianzas.max(), inplace=True)
# Calculamos el peso como la inversa de la varianza
df_sell_in_grouped['peso2'] = 1 / varianzas

#Opcion 3: Ventas recientes fuertes (exponencial decay)
pesos_temporales = np.exp(-np.arange(len(lags)))  # tn tiene peso más alto
df_sell_in_grouped['peso3'] = df_sell_in_grouped[lags].multiply(pesos_temporales, axis=1).sum(axis=1)

df_sell_in_grouped

,product_id,periodo,tn,clase,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11,tn_actual,peso1,peso2,peso3
0,20001,201701,934.77222,1303.35771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,934.77222,934.772220,0.000005,934.772220
1,20001,201702,798.01620,1069.96130,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,798.01620,866.394210,0.000107,1141.899682
2,20001,201703,1303.35771,1502.20132,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1303.35771,1012.048710,0.000015,1723.439127
3,20001,201704,1069.96130,1520.06539,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1069.96130,1026.526858,0.000022,1703.979123
4,20001,201705,1502.20132,1030.67391,1069.96130,1303.35771,798.01620,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1502.20132,1121.661750,0.000012,2129.060207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31238,21295,201701,0.00699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00699,0.006990,0.000005,0.006990
31239,21296,201708,0.00651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00651,0.006510,0.000005,0.006510
31240,21297,201701,0.00579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00579,0.005790,0.000005,0.005790
31241,21298,201708,0.00573,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00573,0.005730,0.000005,0.005730


In [14]:
# Filtrar registros del período 201812
df_201812 = df_sell_in_grouped[df_sell_in_grouped['periodo'] == 201812].copy()

# Crear lista de nombres de los campos tn_1 a tn_11
lag_features = [f'tn_{i}' for i in range(1, 11)]

# Filtrar los registros que NO tienen nulos en ningún lag
completos = df_201812.dropna(subset=lag_features)

# Unir con dr_productos para obtener cat1, cat2 y cat3
completos = completos.merge(df_productos[['product_id', 'cat1', 'cat2', 'brand']], on='product_id', how='left')

# Mostrar los resultados ordenados por product_id
print(completos[['product_id', 'cat1', 'cat2', 'brand']].drop_duplicates().sort_values(by='product_id').to_string(index=False))


 product_id  cat1                cat2     brand
      20001    HC         ROPA LAVADO     ARIEL
      20002    HC         ROPA LAVADO   LIMPIEX
      20003 FOODS            ADEREZOS    NATURA
      20004 FOODS            ADEREZOS    NATURA
      20005 FOODS            ADEREZOS    NATURA
      20006    HC             VAJILLA   LIMPIEX
      20007    HC ROPA ACONDICIONADOR     DOWNY
      20008    HC             VAJILLA   LIMPIEX
      20009    HC         ROPA LAVADO    ROPEX1
      20010    HC         ROPA LAVADO   LIMPIEX
      20011    HC         ROPA LAVADO   LIMPIEX
      20012    HC ROPA ACONDICIONADOR     DOWNY
      20013    HC               HOGAR   MUSCULO
      20014    HC ROPA ACONDICIONADOR     DOWNY
      20015    HC         ROPA LAVADO    ROPEX1
      20016    HC         ROPA LAVADO   LIMPIEX
      20017    HC             VAJILLA   LIMPIEX
      20018    HC             VAJILLA   LIMPIEX
      20019 FOODS            ADEREZOS    NATURA
      20020    HC         ROPA LAVADO   

In [15]:
# Paso 1: Filtrar al período 201812
df_201812 = df_sell_in_grouped[df_sell_in_grouped['periodo'] == 201812].copy()

# Paso 2: Verificar que todos los lags estén presentes (tn_1 a tn_11)
lag_cols = [f"tn_{i}" for i in range(1, 12)]
df_completos = df_201812[df_201812[lag_cols].notnull().all(axis=1)].copy()

# Paso 3: Calcular estadísticas por producto
df_completos["media_tn"] = df_completos[lag_cols].mean(axis=1)
df_completos["std_tn"] = df_completos[lag_cols].std(axis=1)
df_completos["coef_var"] = df_completos["std_tn"] / df_completos["media_tn"]

# Paso 4: Aplicar umbrales
umbral_volumen = 50      # promedio de ventas (en toneladas) de los últimos 11 meses del producto
umbral_cv = 0.75         # umbral para el coeficiente de variación. Es una medida de estabilidad. Cuanto menor el coeficiente de variación, más estable es la serie temporal del producto.
df_filtrado = df_completos[
    (df_completos["media_tn"] >= umbral_volumen) &
    (df_completos["coef_var"] <= umbral_cv)
].copy()

# Paso 5: Agregar información de categorías y marca
df_filtrado = df_filtrado.merge(
    df_productos[['product_id', 'cat1', 'cat2', 'cat3', 'brand']],
    on='product_id',
    how='left'
)

# Paso 6: Mostrar los candidatos a productos mágicos
print("\n✅ Candidatos a productos mágicos:\n")
print(df_filtrado[['product_id', 'cat1', 'cat2', 'cat3', 'brand', 'media_tn', 'coef_var']].sort_values(by='media_tn', ascending=False).to_string(index=False))


✅ Candidatos a productos mágicos:

 product_id  cat1                cat2             cat3    brand    media_tn  coef_var
      20001    HC         ROPA LAVADO          Liquido    ARIEL 1507.628118  0.256564
      20002    HC         ROPA LAVADO          Liquido  LIMPIEX 1094.446920  0.251567
      20003 FOODS            ADEREZOS         Mayonesa   NATURA  884.526920  0.237449
      20004 FOODS            ADEREZOS         Mayonesa   NATURA  659.959564  0.288989
      20007    HC ROPA ACONDICIONADOR   ACONDICIONADOR    DOWNY  655.480846  0.342175
      20006    HC             VAJILLA       Cristalino  LIMPIEX  608.341486  0.246947
      20005 FOODS            ADEREZOS         Mayonesa   NATURA  596.353755  0.292641
      20008    HC             VAJILLA            Opaco  LIMPIEX  561.759514  0.199567
      20012    HC ROPA ACONDICIONADOR   ACONDICIONADOR    DOWNY  514.807323  0.308037
      20014    HC ROPA ACONDICIONADOR   ACONDICIONADOR    DOWNY  467.611692  0.180948
      20016    HC 

In [16]:
# Seleccionamos los top 40
mejores_40 = df_filtrado.head(40)

# Mostramos cuántos productos fueron seleccionados y extraemos los IDs
print(f"🔢 Total de productos seleccionados: {mejores_40.shape[0]}")

magicos_mejores_40 = mejores_40['product_id'].tolist()
print(f"🧙‍♂️ Lista de productos mágicos:\n{magicos_mejores_40}")

🔢 Total de productos seleccionados: 40
🧙‍♂️ Lista de productos mágicos:
[20001, 20002, 20003, 20004, 20005, 20006, 20007, 20008, 20009, 20010, 20011, 20012, 20013, 20014, 20015, 20016, 20017, 20018, 20019, 20020, 20021, 20022, 20023, 20024, 20025, 20026, 20027, 20028, 20029, 20030, 20031, 20033, 20034, 20035, 20036, 20037, 20038, 20039, 20040, 20041]


In [18]:
# Agrupar por cat1, cat2 y brand
grupos = df_filtrado.groupby(['cat1', 'cat2', 'brand'])

# Lista para guardar los productos seleccionados
productos_seleccionados = []

for _, grupo in grupos:
    # Ordenar por mayor media_tn y, si hay empate, por menor coef_var
    grupo_ordenado = grupo.sort_values(by=['media_tn', 'coef_var'], ascending=[False, True])
    # Tomar el primero (mejor producto del grupo)
    productos_seleccionados.append(grupo_ordenado.iloc[0])

# Crear nuevo DataFrame con los seleccionados
mejores_productos = pd.DataFrame(productos_seleccionados)

# Mostrar resultado
print(mejores_productos[['product_id', 'cat1', 'cat2', 'brand', 'media_tn', 'coef_var']].sort_values(by='media_tn', ascending=False).to_string(index=False))

# Contar cuántos productos fueron seleccionados
cantidad_productos = mejores_productos.shape[0]
print(f"🔢 Total de productos seleccionados: {cantidad_productos}")

# Extraer los IDs de producto como lista
magicos_mejores_filtrados = mejores_productos['product_id'].tolist()
print(f"🧙‍♂️ Lista de productos mágicos:\n{magicos_mejores_filtrados}")

 product_id  cat1                cat2    brand    media_tn  coef_var
      20001    HC         ROPA LAVADO    ARIEL 1507.628118  0.256564
      20002    HC         ROPA LAVADO  LIMPIEX 1094.446920  0.251567
      20003 FOODS            ADEREZOS   NATURA  884.526920  0.237449
      20007    HC ROPA ACONDICIONADOR    DOWNY  655.480846  0.342175
      20006    HC             VAJILLA  LIMPIEX  608.341486  0.246947
      20009    HC         ROPA LAVADO   ROPEX1  444.238566  0.251856
      20013    HC               HOGAR  MUSCULO  416.642425  0.223191
      20023 FOODS            ADEREZOS MOSTAZA1  285.201363  0.268588
      20048 FOODS      SOPAS Y CALDOS    MAGGI  229.604244  0.269671
      20056    HC ROPA ACONDICIONADOR   VIVERE  188.095002  0.427920
      20039    HC        ROPA MANCHAS  LIMPIEX  179.677093  0.214439
      20047    PC             CABELLO SHAMPOO2  163.881225  0.286351
      20149    HC         ROPA LAVADO   ROPEX2  156.958700  0.511332
      20042    HC             VAJI

In [19]:
# Filtrar solo los meses 201812 y 201912
df_base = df_sell_in_grouped[df_sell_in_grouped['periodo'].isin([201812, 201912])]

# Pivotear para tener columnas por periodo
df_pivot = df_base.pivot(index='product_id', columns='periodo', values='tn').dropna()

# Calcular la correlación entre las dos columnas
correlacion = df_pivot.corr().loc[201812, 201912]
print(f"📈 Correlación global tn_201812 vs tn_201912: {correlacion:.3f}")

# Calcular la correlación por producto: ordenar por mayor incidencia
df_pivot['corr'] = (df_pivot[201812] - df_pivot[201812].mean()) * (df_pivot[201912] - df_pivot[201912].mean())
df_pivot['corr'] /= (df_pivot[201812].std() * df_pivot[201912].std())

# Ordenar por mayor correlación positiva
productos_predictivos = df_pivot.sort_values(by='corr', ascending=False).reset_index()

# Seleccionar los top 40
magicos_corr = productos_predictivos.head(40)['product_id'].tolist()
print(f"🧙 Productos mágicos por correlación:\n{magicos_corr}")


📈 Correlación global tn_201812 vs tn_201912: 0.966
🧙 Productos mágicos por correlación:
[20001, 20002, 20003, 20004, 20009, 20005, 20006, 20019, 20007, 20015, 20011, 20013, 20010, 20014, 20008, 20016, 20017, 20012, 20022, 20026, 20020, 20024, 20021, 20018, 20023, 20027, 20045, 20049, 20025, 20028, 20035, 20038, 20053, 20029, 20085, 20046, 20051, 20042, 20031, 20069]


In [20]:
# Paso 5: Seleccionar los registros de 201812 para entrenar
dataset_training = df_sell_in_grouped[df_sell_in_grouped['periodo'] == 201812].copy()

# Lista de productos mágicos del profe
magicos_profe = [20002, 20003, 20006, 20010, 20011, 20018, 20019, 20021,
           20026, 20028, 20035, 20039, 20042, 20044, 20045, 20046, 20049,
           20051, 20052, 20053, 20055, 20008, 20001, 20017, 20086, 20180,
           20193, 20320, 20532, 20612, 20637, 20807, 20838]

In [21]:
# Filtrar los productos mágicos en dr_productos
# magicos_auto es el conjunto alternativo

magicos= magicos_profe #0.25
#magicos= magicos_mejores_40 #1.215
#magicos= magicos_mejores_filtrados #0.47
#magicos= magicos_corr #0.82

magicos_info = df_productos[df_productos['product_id'].isin(magicos)][['product_id', 'cat1', 'cat2', 'brand']]

# Imprimir el resultado ordenado por product_id
print(magicos_info.sort_values(by='product_id').to_string(index=False))


 product_id  cat1                cat2    brand
      20001    HC         ROPA LAVADO    ARIEL
      20002    HC         ROPA LAVADO  LIMPIEX
      20003 FOODS            ADEREZOS   NATURA
      20006    HC             VAJILLA  LIMPIEX
      20008    HC             VAJILLA  LIMPIEX
      20010    HC         ROPA LAVADO  LIMPIEX
      20011    HC         ROPA LAVADO  LIMPIEX
      20017    HC             VAJILLA  LIMPIEX
      20018    HC             VAJILLA  LIMPIEX
      20019 FOODS            ADEREZOS   NATURA
      20021    HC         ROPA LAVADO  LIMPIEX
      20026    HC         ROPA LAVADO  LIMPIEX
      20028    HC         ROPA LAVADO   ROPEX1
      20035    HC         ROPA LAVADO  LIMPIEX
      20039    HC        ROPA MANCHAS  LIMPIEX
      20042    HC             VAJILLA  MUSCULO
      20044    PC             CABELLO SHAMPOO2
      20045    HC         ROPA LAVADO    ARIEL
      20046 FOODS            ADEREZOS   NATURA
      20049    HC ROPA ACONDICIONADOR    DOWNY
      20051  

In [22]:
# Filtrar dataset de entrenamiento
train_df = dataset_training[dataset_training['product_id'].isin(magicos)].dropna()

# Entrenar modelo de regresión lineal
features = ['tn_actual'] + [f'tn_{i}' for i in range(1, 12)]
X_train = train_df[features]
y_train = train_df['clase']
sample_weight = train_df['peso3']

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error
import numpy as np

# Modelos base
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(max_iter=10000),
    "ElasticNet": ElasticNet(max_iter=10000)
}

# Parámetros para búsqueda
param_grids = {
    "LinearRegression": {},  # Sin hiperparámetros
    "Ridge": {
        "alpha": [1.0, 10.0, 100.0, 200.0, 300.0, 350.0, 400.0, 450.0],
        "solver": ['sparse_cg', 'lsqr', 'saga', 'sag']
    },
    "Lasso": {
        "alpha": [1.0, 2.0, 5.0, 50.00, 100.0, 200.0, 300.0, 350.0, 400.0, 450.0]
    },
    "ElasticNet": {
        "alpha": [1.0, 10.0, 50.0, 100.0, 200.0, 300.0, 350.0, 400.0, 450.0],
        "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9]  # mezcla entre L1 (Lasso) y L2 (Ridge)
    }
}

# Evaluación
for name in models:
    print(f"\n🔍 Evaluando modelo: {name}")
    if param_grids[name]:
        grid = GridSearchCV(models[name], param_grids[name], cv=5, scoring='neg_mean_absolute_error', return_train_score=True)
        grid.fit(X_train, y_train)

        results_df = pd.DataFrame(grid.cv_results_)
        results_df['mean_MAE'] = -results_df['mean_test_score']

        # Mostrar todas las combinaciones ordenadas por mejor MAE
        print(results_df[['params', 'mean_MAE']].sort_values('mean_MAE'))

        # Mostrar el mejor resultado
        print(f"\n✅ Mejor combinación: {grid.best_params_}")
        print(f"📊 MAE promedio (CV): {-grid.best_score_:.2f}")
    else:
        models[name].fit(X_train, y_train)
        preds = models[name].predict(X_train)
        mae = mean_absolute_error(y_train, preds)
        print(f"✅ Sin hiperparámetros. MAE entrenamiento: {mae:.2f}")



🔍 Evaluando modelo: LinearRegression
✅ Sin hiperparámetros. MAE entrenamiento: 21.02

🔍 Evaluando modelo: Ridge


/Users/matias/dev/laboratorio3/labo3-2025v/labo3/.venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/matias/dev/laboratorio3/labo3-2025v/labo3/.venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/matias/dev/laboratorio3/labo3-2025v/labo3/.venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/matias/dev/laboratorio3/labo3-2025v/labo3/.venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/matias/dev/laboratorio3/labo3-2025v/labo3/.venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: Conver

                                     params   mean_MAE
30       {'alpha': 450.0, 'solver': 'saga'}  43.974905
26       {'alpha': 400.0, 'solver': 'saga'}  44.006560
22       {'alpha': 350.0, 'solver': 'saga'}  44.110012
18       {'alpha': 300.0, 'solver': 'saga'}  44.177561
14       {'alpha': 200.0, 'solver': 'saga'}  44.316606
10       {'alpha': 100.0, 'solver': 'saga'}  44.461966
2          {'alpha': 1.0, 'solver': 'saga'}  44.601763
6         {'alpha': 10.0, 'solver': 'saga'}  44.605619
31        {'alpha': 450.0, 'solver': 'sag'}  45.364947
27        {'alpha': 400.0, 'solver': 'sag'}  45.445338
23        {'alpha': 350.0, 'solver': 'sag'}  45.548052
19        {'alpha': 300.0, 'solver': 'sag'}  45.681574
15        {'alpha': 200.0, 'solver': 'sag'}  45.897878
11        {'alpha': 100.0, 'solver': 'sag'}  46.110050
7          {'alpha': 10.0, 'solver': 'sag'}  46.298250
3           {'alpha': 1.0, 'solver': 'sag'}  46.350440
29       {'alpha': 450.0, 'solver': 'lsqr'}  49.164779
28  {'alph

In [24]:
#model = Lasso(max_iter=10000, alpha=450.0)
#model = ElasticNet(alpha=450.0, l1_ratio=0.9, max_iter=10000)

model = LinearRegression()
model.fit(X_train, y_train)
#model.fit(X_train, y_train, sample_weight=sample_weight) #se agrega ponderacion por pesos, anduvo mejor el peso1

# Guardamos los coeficientes
coef_df = pd.DataFrame({
    'feature': ['intercept'] + features,
    'coeficiente': [model.intercept_] + list(model.coef_)
})

# Mostrar coeficientes
coef_df

,feature,coeficiente
0,intercept,0.441467
1,tn_actual,-0.001339
2,tn_1,0.236558
3,tn_2,0.178208
4,tn_3,-0.060031
5,tn_4,-0.161875
6,tn_5,-0.007775
7,tn_6,0.151936
8,tn_7,0.043933
9,tn_8,0.142839


In [25]:
# Paso 6: Aplicar el modelo SOLO a productos del dataset a predecir
predict_df = df_sell_in_grouped[
    (df_sell_in_grouped['periodo'] == 201912) & 
    (df_sell_in_grouped['product_id'].isin(productos_a_predecir))
].copy()
predict_df['completos'] = predict_df[features].notnull().all(axis=1)
predict_df

,product_id,periodo,tn,clase,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11,tn_actual,peso1,peso2,peso3,completos
35,20001,201912,1504.68856,NaN,1397.37231,1561.50552,1660.00561,1261.34529,1678.99318,1109.93769,1629.78233,1647.63848,1470.65653,1259.09363,1275.77351,1504.68856,1454.732720,0.000027,2352.192136,True
71,20002,201912,1087.30855,NaN,1423.57739,1979.53635,1090.18771,813.78215,1066.44999,928.36431,1034.98927,1287.62346,1083.62552,1043.01349,1266.78751,1087.30855,1175.437142,0.000011,1959.161554,True
107,20003,201912,892.50129,NaN,948.29393,1081.36645,967.77116,635.59563,715.20314,662.38654,590.12515,565.33774,638.04010,758.32657,964.76919,892.50129,784.976407,0.000032,1454.847944,True
143,20004,201912,637.90002,NaN,723.94206,1064.69633,786.17140,482.13372,521.71519,667.19411,603.31081,466.70901,619.77084,441.70332,511.33713,637.90002,627.215328,0.000033,1102.267045,True
179,20005,201912,593.24443,NaN,606.91173,996.78275,879.52808,536.66800,745.74978,876.39696,897.26297,624.99880,488.21387,409.89950,363.58438,593.24443,668.270104,0.000023,1013.343183,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31090,21263,201912,0.01270,NaN,0.03247,0.01552,0.01128,0.03388,0.03387,0.00988,0.02258,0.01835,0.06636,0.05927,0.04376,0.01270,0.029993,2850.299276,0.028219,True
31129,21265,201912,0.05007,NaN,0.06600,0.10921,0.01707,0.01593,0.02959,0.05121,0.17635,0.36405,0.01593,NaN,NaN,0.05007,0.089541,84.473258,0.090883,False
31139,21266,201912,0.05121,NaN,0.06713,0.11831,0.02844,0.01480,0.05916,0.05235,0.17634,0.36178,0.01707,NaN,NaN,0.05121,0.094659,89.053235,0.094417,False
31149,21267,201912,0.01569,NaN,0.04052,0.09676,0.01830,0.04054,0.07452,0.05882,0.24451,0.12291,0.21578,NaN,NaN,0.01569,0.092835,156.490188,0.046284,False


In [26]:
# Aplicar modelo a los productos completos
X_pred = predict_df.loc[predict_df['completos'], features]
predict_df.loc[predict_df['completos'], 'tn_pred'] = model.predict(X_pred)

# 🔍 Ver predicciones para productos con datos completos
print("🔍 Predicciones para productos con features completas:")
print(predict_df.loc[predict_df['completos'], ['product_id', 'tn_pred']])

🔍 Predicciones para productos con features completas:
       product_id      tn_pred
35          20001  1162.707525
71          20002  1183.640604
107         20003   684.763931
143         20004   580.484961
179         20005   563.560780
...           ...          ...
30913       21248     0.468061
30985       21256     0.463856
31036       21259     0.467856
31075       21262     0.465820
31090       21263     0.467764

[656 rows x 2 columns]


In [27]:
# Para los incompletos: usar promedio histórico por producto
# Primero obtenemos el promedio histórico de tn para cada producto (sin incluir 201912 ni posteriores)
promedios_historicos = df_sell_in_grouped[df_sell_in_grouped['periodo'] < 201912] \
    .groupby('product_id')['tn'].mean().to_dict()
promedios_historicos

{20001: 1395.3059148571429,
 20002: 1007.1413097142857,
 20003: 888.9043271428571,
 20004: 672.5786791428571,
 20005: 645.6564025714285,
 20006: 590.6150797142857,
 20007: 617.943372,
 20008: 564.3692851428572,
 20009: 468.45681571428577,
 20010: 523.1851200000001,
 20011: 448.37208000000004,
 20012: 504.01340885714285,
 20013: 446.35008600000003,
 20014: 454.44874942857143,
 20015: 415.2310602857143,
 20016: 402.2343,
 20017: 379.4023057142857,
 20018: 372.4906534285714,
 20019: 383.2850245714286,
 20020: 342.6378988571429,
 20021: 351.3749728571429,
 20022: 299.5189897142857,
 20023: 292.975878,
 20024: 296.236044,
 20025: 259.287912,
 20026: 263.0888591428572,
 20027: 236.2439114285714,
 20028: 260.916552,
 20029: 245.6408597142857,
 20030: 280.57744,
 20031: 241.67569457142858,
 20032: 596.587215,
 20033: 221.61765599999998,
 20034: 314.42447400000003,
 20035: 214.50431885714286,
 20036: 285.78151875,
 20037: 173.10332742857145,
 20038: 197.75956714285715,
 20039: 181.7765168571428

In [28]:
promedio_global = df_sell_in_grouped[df_sell_in_grouped['periodo'] < 201912]['tn'].mean()
# Función para asignar el promedio correspondiente si tn_pred está en NaN
def imputar_promedio_individual(row):
    if pd.isna(row['tn_pred']):
        return promedios_historicos.get(row['product_id'], promedio_global)
        #return promedios_historicos.get(row['product_id'], 0.0)
    return row['tn_pred']

# Aplicar imputación
mask_incompletos = predict_df['completos'] == False
predict_df.loc[mask_incompletos, 'tn_pred'] = predict_df[mask_incompletos].apply(imputar_promedio_individual, axis=1)

# Verificación: mostrar si efectivamente se usó el promedio específico
predict_df['promedio_historico'] = predict_df['product_id'].map(promedios_historicos)
predict_df['usó_promedio_historico'] = (~predict_df['completos']) & (predict_df['tn_pred'] == predict_df['promedio_historico'])

# Mostrar productos donde se usó el promedio histórico
print(predict_df.loc[~predict_df['completos'], ['product_id', 'tn_pred', 'promedio_historico', 'usó_promedio_historico']])

       product_id     tn_pred  promedio_historico  usó_promedio_historico
1126        20032  596.587215          596.587215                    True
4351        20127  221.140627          221.140627                    True
5834        20174  118.760752          118.760752                    True
6990        20210   89.975557           89.975557                    True
7067        20213   82.940152           82.940152                    True
...           ...         ...                 ...                     ...
30929       21252    0.188279            0.188279                    True
31129       21265    0.093927            0.093927                    True
31139       21266    0.099487            0.099487                    True
31149       21267    0.101407            0.101407                    True
31210       21276    0.049506            0.049506                    True

[124 rows x 4 columns]


In [30]:
# Resultado final: predict_df con columnas product_id y tn_pred para febrero 2020
submission_df = predict_df[['product_id', 'tn_pred']].copy()
submission_df.rename(columns={'tn_pred': 'tn'}, inplace=True)

# Verificamos que no haya duplicados y haya exactamente 780 registros
assert submission_df.shape[0] == 780, "❌ El archivo final no tiene 780 registros"
assert submission_df['product_id'].duplicated().sum() == 0, "❌ Hay productos duplicados en el archivo final"

submission_df.to_csv('../data/kaggle_submission_202002_regresion.csv', index=False)

print("✅ Predicción generada y archivo CSV listo para subir a Kaggle.")

✅ Predicción generada y archivo CSV listo para subir a Kaggle.


In [31]:
import json

# Cargar y mostrar el contenido del archivo kaggle.json
try:
    with open("kaggle.json", "r") as f:
        creds = json.load(f)
        print("✅ Archivo kaggle.json leído correctamente.")
        print(f"👤 Usuario: {creds.get('username')}")
        print(f"🔑 API Key: {creds.get('key')[:4]}... (oculto por seguridad)")
except FileNotFoundError:
    print("❌ El archivo kaggle.json no se encuentra en el directorio actual.")
except json.JSONDecodeError:
    print("❌ El archivo kaggle.json no tiene un formato JSON válido.")
except Exception as e:
    print(f"❌ Error al leer kaggle.json: {e}")

❌ El archivo kaggle.json no se encuentra en el directorio actual.


In [1018]:
import subprocess

try:
    result = subprocess.run(
        [
            "kaggle", "competitions", "submit",
            "-c", "labo-iii-edicion-2025-v",
            "-f", "kaggle_submission_202002_regresion.csv",
            "-m", "Predicción febrero 2020"
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        encoding="utf-8"  # cambia cp1252 (default en Windows) a utf-8
    )

    if result.returncode == 0:
        print("✅ Subida a Kaggle completada con éxito:")
        print(result.stdout)
    else:
        print("❌ Error al subir a Kaggle:")
        print(result.stderr)

except FileNotFoundError:
    print("❌ El comando 'kaggle' no se encontró. Verificá que esté instalado y en el PATH.")


✅ Subida a Kaggle completada con éxito:
Successfully submitted to Labo III, edicion 2025v
